# TMPSF Examples

## Download data from Google Drive if necessary

In [ ]:
blob_ids = ['1w-f9wRHqHr5Uqe7MXwC2XKAD1cupn5yF',
            '1XgYcdx5ZvQIkhtWGZU73-MJ6sWAv2YuS',
            '1vB0g0ksFty4wzR1YP38OcqemmS5QXax_',
            '1kyKo24QAcIc8qFXLwMZhHBaAI3JLdJis',
            '1z2gjfFcpqPbcfyhWam6FNdDG7ltXYKn1',
            '1RNGWfEoJU5DhWt9df72PHH9FmFT86xa0']

In [ ]:
file_list = ['deployment0001_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20140929T190312-20141231T235952.nc',
             'deployment0001_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20150101T000002-20151231T235958.777243.nc',
             'deployment0001_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20160101T000010.778112-20161231T235958.439167.nc',
             'deployment0001_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20170101T000010.438824-20170813T232226.349596.nc',
             'deployment0003_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20170815T003130.804600-20171231T235950.623028.nc',
             'deployment0003_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20180101T000000.621938-20180912T225325.872315.nc']

In [ ]:
import driveanon as da
from pathlib import Path

In [ ]:
for i, blob_id in enumerate(blob_ids):
    if not Path(file_list[i]).is_file():
        da.save(blob_id)
        print(file_list[i])

## Open NetCDF files as xarray dataset

In [ ]:
import xarray as xr

In [ ]:
ds = xr.open_mfdataset(file_list, drop_variables=['lat', 'lon', 'obs',
                                                  'id', 'battery_voltage',
                                                  'driver_timestamp',
                                                  'ingestion_timestamp',
                                                  'internal_timestamp',
                                                  'port_timestamp',
                                                  'preferred_timestamp',
                                                  'timestamp',
                                                  'provenance', 'serial_number',
                                                  'temperature01_qc_executed',
                                                  'temperature01_qc_results',
                                                  'temperature02_qc_executed',
                                                  'temperature02_qc_results',
                                                  'temperature03_qc_executed',
                                                  'temperature03_qc_results',
                                                  'temperature04_qc_executed',
                                                  'temperature04_qc_results',
                                                  'temperature05_qc_executed',
                                                  'temperature05_qc_results',
                                                  'temperature06_qc_executed',
                                                  'temperature06_qc_results',
                                                  'temperature07_qc_executed',
                                                  'temperature07_qc_results',
                                                  'temperature08_qc_executed',
                                                  'temperature08_qc_results',
                                                  'temperature09_qc_executed',
                                                  'temperature09_qc_results',
                                                  'temperature10_qc_executed',
                                                  'temperature10_qc_results',
                                                  'temperature11_qc_executed',
                                                  'temperature11_qc_results',
                                                  'temperature12_qc_executed',
                                                  'temperature12_qc_results',
                                                  'temperature13_qc_executed',
                                                  'temperature13_qc_results',
                                                  'temperature14_qc_executed',
                                                  'temperature14_qc_results',
                                                  'temperature15_qc_executed',
                                                  'temperature15_qc_results',
                                                  'temperature16_qc_executed',
                                                  'temperature16_qc_results',
                                                  'temperature17_qc_executed',
                                                  'temperature17_qc_results',
                                                  'temperature18_qc_executed',
                                                  'temperature18_qc_results',
                                                  'temperature19_qc_executed',
                                                  'temperature19_qc_results',
                                                  'temperature20_qc_executed',
                                                  'temperature20_qc_results',
                                                  'temperature21_qc_executed',
                                                  'temperature21_qc_results',
                                                  'temperature22_qc_executed',
                                                  'temperature22_qc_results',
                                                  'temperature23_qc_executed',
                                                  'temperature23_qc_results',
                                                  'temperature24_qc_executed',
                                                  'temperature24_qc_results',
                                                  'temperature01_qc_executed',
                                                  'temperature01_qc_results'])
ds = ds.swap_dims({'obs': 'time'})
ds

## Convert Xarray dataset to Pandas dataframe

In [ ]:
import pandas as pd

In [ ]:
df = ds.to_dataframe()
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## Monthly stats

In [ ]:
df['year'] = df.index.year
df['month'] = df.index.month

In [ ]:
df.groupby(['year', 'month']).describe()

## Hourly means

In [ ]:
df['date'] = df.index.date
df['hour'] = df.index.hour

In [ ]:
hourly_means = df.groupby(['date', 'hour']).mean()
hourly_means.head()

## Plot all data for one sensor

In [ ]:
from datetime import datetime
import matplotlib.dates as dates

In [ ]:
dts = [dates.date2num(datetime(x[0].year, x[0].month, x[0].day, x[1])) for x in hourly_means.index]

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt

In [ ]:
plt.rc('font', size=11)
fig, ax = plt.subplots()
fig.set_size_inches(14, 6)
fig.frameon = False
hb1 = ax.hexbin(dts, hourly_means.temperature01, vmin=0, vmax=2, linewidths=0.25,
  gridsize=(225,4500), mincnt=1, cmap=plt.cm.inferno)
fig.colorbar(hb1)
#ax.set_ylim([0, 8000])
#ax.set_xlim([frame_timestamp[0],frame_timestamp[-1]])
#ax.yaxis.grid(True)
#ax.xaxis.grid(True)
tick_loc = dates.YearLocator()  # every year
tick_fmt = dates.DateFormatter('%b %Y')
ax.xaxis.set_major_locator(tick_loc)
ax.xaxis.set_major_formatter(tick_fmt)
#plt.ylabel('Floc Proxy Value');

In [ ]:
len(hourly_means.temperature01)